# artsy

In [1]:
import json

from pymongo import MongoClient
import requests

import credentials

## setup

### database

In [2]:
client = MongoClient()
client.drop_database('artsy')
db = client.artsy

### base url

In [3]:
BASE_URL = 'https://api.artsy.net/api'

### app

In [4]:
app_name = credentials.artsy_app_name
client_id = credentials.artsy_client_id
client_secret = credentials.artsy_client_secret

### token

In [5]:
headers = {}
payload = {'client_id': client_id, 'client_secret': client_secret}
url = BASE_URL + '/tokens/xapp_token'
response = requests.post(url, params=payload, headers=headers)
token = response.json()['token']

### auxiliar

In [6]:
def change_id_field(d):
    d['_id'] = d.pop('id')

### test

In [7]:
# headers = {'X-Xapp-Token': token}x
# payload = {'total_count': 1}
# url = BASE_URL + '/'
# response = requests.get(url, params=payload, headers=headers)
# # total_count = response.json()['total_count']
# print(response.json())

## artsy resources
* Status
* Artists
* Artworks
* Editions
* Genes
* Profiles
* Partners
* Shows
* Fairs
* Applications
* Users
* User Details
* Collections
* Collection Items
* Images
* Search

## status

In [ ]:
headers = {}
payload = {'total_count': 1}
url = BASE_URL + '/status'
response = requests.get(url, params=payload, headers=headers)
status = response.json()['status']
print('Status: %s' % status)

Status: good


## resources
artists, artworks, genes, profiles, partners, shows, fairs

In [ ]:
resources = ['artists', 'artworks', 'genes', 'profiles', 'partners', 'shows', 'fairs']
for resource in resources:
    print('Processing %s...' % resource)
    url = BASE_URL + '/' + resource
    headers = {'X-Xapp-Token': token}
    
    # total count
    payload = {'total_count': 1}
    response = requests.get(url, params=payload, headers=headers)
    total_count = response.json()['total_count']
    print('Total %s: %d' % (resource, total_count))
    
    # data
    payload = {}
    count = 0
    while True:
        response = requests.get(url, params=payload, headers=headers)
        data = response.json()['_embedded'][resource]
        if not data:
            break
        [change_id_field(d) for d in data]
        db[resource].insert_many(data)
        url = response.json()['_links']['next']['href']
        if not url:
            break
        count += len(data)
        if count % (total_count / 10) < len(data):
            print('%d %s processed.' % (resource, count))

Processing artists...
Total artists: 49934

## search

In [ ]:
search_term = 'Andy Warhol'

In [ ]:
search_url = BASE_URL + '/search'
headers = {'X-Xapp-Token': token}
payload = {'q': search_term}
response = requests.get(search_url, params=payload, headers=headers)

In [ ]:
response

In [ ]:
response.json()